# ResNet CIFAR10 Demo

Tested on `Python3.9.17`


To install the package:
```bash
python3.9 -m venv venv
source venv/bin/activate  # For Linux only, activate your virtual env accordingly based on your OS
pip install --upgrade pip
pip install -r requirements.txt
```

In [ ]:
import torch
from torch.utils.data import DataLoader
import torchvision as tv
import torchvision.transforms as T
import detectors
import numpy as np

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

The hyperparameters for the pre-trained model can be found here:
[https://huggingface.co/edadaltocg/resnet18_cifar10](https://huggingface.co/edadaltocg/resnet18_cifar10)

In [ ]:
# Download pre-trained model for ResNet18 trained on CIFAR10:
model = detectors.create_model("resnet18_cifar10", pretrained=True)
model = model.to(device)  # Move model to GPU

# NOTE: Since we load a pre-trained model, we do NOT need define our transformation
# # Additional transformations are only applied to training set to avoid overfitting
# transform_train = T.Compose([
#     T.Pad(4),
#     T.RandomCrop(32, fill=128),
#     T.RandomHorizontalFlip(),
#     T.ToTensor(),
#     T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
# ])
# # Only normalize the range (From 0-255 to 0-1)
# transform_test = T.Compose([
#     T.ToTensor(),
#     T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
# ])

transform_train = detectors.create_transform(model, is_training=True)
transform_test = detectors.create_transform(model)

In [ ]:
# Almost identical to our transform functions:
print(transform_train)
print(transform_test)

In [ ]:
# Preparing data
# Hyperparameters:
BATCH_SIZE = 64  # Based on GPU's VRAM
NUM_THREADS = 8  # Based on # of CPU cores

# NOTE: We use `transform_test` for training set, because the model is pre-trained, we only interested in its accuracy.
dataset_train = tv.datasets.CIFAR10('./data', download=True, train=True, transform=transform_test)
dataset_test = tv.datasets.CIFAR10('./data', download=True, train=False, transform=transform_test)

# NOTE: Evaluation only. Turn shuffle off.
dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_THREADS)
dataloader_test = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_THREADS)

In [ ]:
def evaluation(model, dataloader, device):
    """This function returns the accuracy of a given dataset on a pre-trained model."""
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            x, y = batch
            x = x.to(device)
            outputs = model(x)
            _, predictions = torch.max(outputs, 1)
            predictions = predictions.to('cpu')
            total += y.size(0)
            correct += (predictions == y).sum().item()
    accuracy = correct / total
    return accuracy

In [ ]:
acc_train = evaluation(model, dataloader_train, device)
acc_test = evaluation(model, dataloader_test, device)

print(f'Train accuracy: {acc_train * 100:.2f}')
print(f'Test accuracy:  {acc_test * 100:.2f}')

## Attack the pre-trained model

In [22]:
from torch.utils.data import TensorDataset

from art.estimators.classification import PyTorchClassifier
import art.attacks.evasion as evasion


In [ ]:
# Checking the image size
(x, y) = next(iter(dataset_test))
input_shape = np.array(x.size())
print(f'input_shape: {input_shape}')

# Check the clip values:
global_min = 9999.
global_max = 0.
for batch in dataloader_train:
    x, _ = batch
    global_min = min(torch.min(x).item(), global_min)
    global_max = max(torch.max(x).item(), global_max)

print(f'Min: {global_min}, Max: {global_max}')

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0005, nesterov=True)
loss_fn = torch.nn.CrossEntropyLoss()

# Images should be within [0, 1] range, but it is not due to the normalization.
classifier = PyTorchClassifier(
    model=model,
    clip_values=(global_min, global_max), 
    loss=criterion,
    optimizer=optimizer,
    input_shape=input_shape,
    nb_classes=10,
)

In [29]:
attack = evasion.AutoProjectedGradientDescent(estimator=classifier)

In [30]:
# Only test with one batch
batch = next(iter(dataloader_test))
X, y = batch
X_advx = attack.generate(x=X.numpy())

AutoPGD - restart:   0%|          | 0/5 [00:00<?, ?it/s]











































AutoPGD - restart:  20%|██        | 1/5 [00:04<00:17,  4.43s/it]


In [31]:
dataset_advx = TensorDataset(torch.Tensor(X_advx), y)
dataloader_advx = DataLoader(dataset_advx, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_THREADS)
acc_advx = evaluation(model, dataloader_advx, device)

print(f'Adversarial examples accuracy: {acc_advx * 100:.2f}')

Adversarial examples accuracy: 3.12
